# Small Caps Invest

In [2]:
import os
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import yfinance as yf   


## Enregistrement manuel des trades

In [19]:
# Enregistrer un achat ou une vente 

csv_file = "transactions.csv"

# Vérifier si le fichier n'existe pas ou s'il est vide pour créer les en-têtes
if not os.path.exists(csv_file) or os.path.getsize(csv_file) == 0:
    headers = "date,ticker,quantite,prix,montant,total_compte,montant_invest,montant_libre\n"
    with open(csv_file, "w") as f:
        f.write(headers)

# demande les détails de la transaction à l'utilisateur
invest = float(input("Entrer un montant investi (ou 0 si aucun) : "))
ticker = input("Entrer le ticker de l'action : ")
quantite = float(int(input("Entrer la quantité achetée (positif) ou vendue (négatif) : ")))
prix = float(input("Entrer le prix unitaire de l'action : "))
date = input("Entrer la date de la transaction (YYYY-MM-DD) : ")
date = datetime.strptime(date, "%Y-%m-%d").date() #convertir en format date
montant = quantite * prix

# Calcul du total_compte et PnL
df = pd.read_csv(csv_file)
if len(df) > 0:
    total_compte = montant + df.iloc[-1]["total_compte"]
    montant_invest = invest + df.iloc[-1]["montant_invest"]
    montant_libre = montant_invest - total_compte
else:
    total_compte = montant
    montant_invest = invest
    montant_libre = montant_invest - total_compte

# Enregistrer l'achat ou la vente dans le fichier CSV   
with open("transactions.csv", "a") as f:
    f.write(f"{date.isoformat()},{ticker},{quantite},{prix},{montant},{total_compte},{montant_invest},{montant_libre}\n")

# Afficher le contenu du fichier CSV
df = pd.read_csv("transactions.csv")
df.head()


,date,ticker,quantite,prix,montant,total_compte,montant_invest,montant_libre
0,2025-06-10,MBOT,6.0,3.4,20.4,20.4,100.0,-79.6
1,2025-06-10,PESI,11.0,2.3,25.3,45.7,100.0,-54.3
2,2025-06-15,F,2.0,11.4,22.8,68.5,100.0,-31.5
3,2025-06-18,MBOT,-3.0,3.1,-9.3,59.2,100.0,-40.8
4,2025-06-20,PESI,-3.0,14.0,-42.0,17.2,100.0,82.8


In [23]:
df['montant_libre'] = df['montant_invest'] - df['total_compte']
df.to_csv("transactions.csv", index=False)

In [31]:
df.head()

,date,ticker,quantite,prix,montant,total_compte,montant_invest,montant_libre,quantite_cum,prix_cloture,valeur_position
0,2025-06-10,MBOT,6.0,3.4,20.4,20.4,100.0,79.6,6.0,2.60,15.599999
1,2025-06-10,PESI,11.0,2.3,25.3,45.7,100.0,54.3,11.0,10.36,113.959996
2,2025-06-15,F,2.0,11.4,22.8,68.5,100.0,31.5,2.0,11.40,22.800000
3,2025-06-18,MBOT,-3.0,3.1,-9.3,59.2,100.0,40.8,3.0,2.47,7.410000
4,2025-06-20,PESI,-3.0,14.0,-42.0,17.2,100.0,82.8,8.0,9.99,79.919998


In [35]:

df = pd.read_csv("transactions.csv")
df['date'] = pd.to_datetime(df['date'])

tickers = df['ticker'].unique()
start = df['date'].min()
end = df['date'].max() + pd.Timedelta(days=1)

closes = []
for ticker in tickers:
    data = yf.download(ticker, start=start, end=end, progress=False, auto_adjust=False)
    if not data.empty and 'Close' in data.columns:
        temp = data[['Close']].reset_index()
        temp['ticker'] = ticker
        closes.append(temp)

df_closes = pd.concat(closes, ignore_index=True)
df_closes.rename(columns={'Date': 'date', 'Close': 'prix_cloture'}, inplace=True)

# Résultat : df_closes contient les colonnes ['date', 'prix_cloture', 'ticker']
df_closes.to_csv("closes.csv", index=False)
df_closes.head()


Price        date prix_cloture ticker prix_cloture    
Ticker                    MBOT                PESI   F
0      2025-06-10         2.60   MBOT          NaN NaN
1      2025-06-11         2.50   MBOT          NaN NaN
2      2025-06-12         2.48   MBOT          NaN NaN
3      2025-06-13         2.34   MBOT          NaN NaN
4      2025-06-16         2.45   MBOT          NaN NaN

In [33]:
#télécharge à partir de yahoo finance les prix de cloture pour chaque ticker et chaque jour

df = pd.read_csv("transactions.csv")
df['date'] = pd.to_datetime(df['date'])

# Liste unique des tickers
tickers = df['ticker'].unique()

# Télécharger les prix de clôture pour chaque ticker sur toute la période
close_prices = {}
for ticker in tickers:
    start = df[df['ticker'] == ticker]['date'].min()
    end = df[df['ticker'] == ticker]['date'].max() + pd.Timedelta(days=1)
    data = yf.download(ticker, start=start, end=end, progress=False, auto_adjust=False)
    if not data.empty and 'Close' in data.columns:
        close_prices[ticker] = data['Close']
    else:
        close_prices[ticker] = pd.Series(dtype='float64')  # Série vide si pas de données

def get_close(row):
    ticker = row['ticker']
    date = row['date']
    try:
        return close_prices[ticker].loc[date]
    except KeyError:
        return row['prix']  # fallback si le prix n'est pas disponible

df['prix_cloture'] = df.apply(get_close, axis=1)

df.head()





1 Failed download:
['F']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-06-15 00:00:00 -> 2025-06-16 00:00:00)')


ValueError: Cannot set a DataFrame with multiple columns to the single column prix_cloture

In [30]:
df = pd.read_csv("transactions.csv")
df['date'] = pd.to_datetime(df['date'])

# Calculer la position cumulée par ticker
df['quantite_cum'] = df.groupby('ticker')['quantite'].cumsum()

# Récupérer le prix de clôture pour chaque ticker et chaque date
def get_close_price(row):
    # S'assurer que row['date'] est bien un Timestamp
    date = pd.to_datetime(row['date'])
    data = yf.download(row['ticker'], start=date, end=date + pd.Timedelta(days=1), progress=False)
    if not data.empty and 'Close' in data.columns:
        return float(data['Close'].iloc[0])
    else:
        return float(row['prix'])  # fallback sur le prix de transaction

df['prix_cloture'] = df.apply(get_close_price, axis=1)

# Calculer la valeur de la position pour chaque transaction
df['valeur_position'] = df['quantite_cum'] * df['prix_cloture']

# Calculer la valeur totale du portfolio chaque jour
portfolio_daily = df.groupby('date').agg(
    taille_portfolio=('quantite_cum', 'sum'),
    valeur_portfolio=('valeur_position', 'sum')
).reset_index()

print(portfolio_daily)

/tmp/ipykernel_8419/4178933434.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(row['ticker'], start=date, end=date + pd.Timedelta(days=1), progress=False)
/tmp/ipykernel_8419/4178933434.py:13: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(data['Close'].iloc[0])
/tmp/ipykernel_8419/4178933434.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(row['ticker'], start=date, end=date + pd.Timedelta(days=1), progress=False)
/tmp/ipykernel_8419/4178933434.py:13: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(data['Close'].iloc[0])
/tmp/ipykernel_8419/4178933434.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download

        date  taille_portfolio  valeur_portfolio
0 2025-06-10              17.0        129.559996
1 2025-06-15               2.0         22.800000
2 2025-06-18               3.0          7.410000
3 2025-06-20               8.0         79.919998


/tmp/ipykernel_8419/4178933434.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(row['ticker'], start=date, end=date + pd.Timedelta(days=1), progress=False)
/tmp/ipykernel_8419/4178933434.py:13: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(data['Close'].iloc[0])
/tmp/ipykernel_8419/4178933434.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(row['ticker'], start=date, end=date + pd.Timedelta(days=1), progress=False)
/tmp/ipykernel_8419/4178933434.py:13: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(data['Close'].iloc[0])


In [ ]:
# mise à jour du portefeuille et du PnL

df = pd.read_csv("transactions.csv")
df['date'] = pd.to_datetime(df['date'])

csv_file = "portfolio.csv"

# Vérifier si le fichier n'existe pas ou s'il est vide pour créer les en-têtes
if not os.path.exists(csv_file) or os.path.getsize(csv_file) == 0:
    headers = "date,ticker,quantite,prix_cloture,montant,total_compte,montant_invest\n"
    with open(csv_file, "w") as f:
        f.write(headers)

# avec les données renseignées mettre à jour le portefeuille
# TOTAL des positions
# valeur du portefeuille et PnL

#

# Calculer pour aujourd'hui le récap des positions et le PnL
# Calculer aussi si montant_invest était dans le S&P500





In [ ]:
# Trace le graphique PnL avec comparatif S&P500


df = pd.read_csv("transactions.csv")    